# Análise de Regressão - Olist Freight Value

## Objetivo
Prever o valor do frete (freight_value) usando diferentes modelos de regressão.

## Requisitos Obrigatórios
- ✅ Regressão Linear
- ✅ PCA para redução de dimensionalidade  
- ✅ 3 Métricas: R², MAE, RMSE

## Modelos Comparados
1. **Linear Regression** (obrigatório)
2. **Random Forest Regressor**
3. **Decision Tree Regressor**


In [1]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics

print("Bibliotecas importadas com sucesso!")


Bibliotecas importadas com sucesso!


## 1. Carregamento dos Dados

In [2]:
# Carregar datasets
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
customers = pd.read_csv('data/olist_customers_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')

print("Dados carregados:")


Dados carregados:


## 2. Features

In [3]:
# Criar feature de volume do produto
products['volume'] = (products['product_length_cm'] * 
                      products['product_height_cm'] * 
                      products['product_width_cm'])

# Preencher valores ausentes na categoria
products['product_category_name'] = products['product_category_name'].fillna('Indefinido')

print("Features criadas")

Features criadas


## 3. Merge e Preparação dos Dados

In [3]:
# Merge dos dados
data = (order_items
        .merge(products, on='product_id')
        .merge(orders, on='order_id')
        .merge(customers, on='customer_id')
        .merge(sellers, on='seller_id')
        .merge(reviews, on='order_id'))

print("Dados unidos")


Dados unidos


## 4. Criação de Features

In [5]:
# Criar feature binária: mesmo estado?
data['same_state'] = (data['customer_state'] == data['seller_state']).astype(int)

# Codificar variáveis categóricas
data['customer_state'] = data['customer_state'].astype('category').cat.codes
data['seller_state'] = data['seller_state'].astype('category').cat.codes
data['product_category'] = data['product_category_name'].astype('category').cat.codes

# Definir features para o modelo
features = [
    'volume', 
    'product_weight_g', 
    'customer_state', 
    'seller_state', 
    'same_state', 
    'product_category'
]

# Preparar X e y (removendo NaN)
X = data[features].dropna()
y = data.loc[X.index, 'freight_value']

print(f"Features criadas: {features}")
print(f"Target (Frete) - média: R$ {y.mean():.2f}")


Features criadas: ['volume', 'product_weight_g', 'customer_state', 'seller_state', 'same_state', 'product_category']
Target (Frete) - média: R$ 19.99


## 5. Train/Test Split e Normalização

In [4]:
# Dividir em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados (necessário para PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Divisão dos dados (20/80)")
print("Normalização aplicada com StandardScaler")

NameError: name 'X' is not defined

## 6. PCA - Redução de Dimensionalidade (Obrigatório)

In [5]:
# Aplicar PCA mantendo 95% da variância
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("PCA aplicado:")
print(f"Dimensões originais: {X_train_scaled.shape[1]}")
print(f"Dimensões após PCA: {X_train_pca.shape[1]}")
print(f"\nComponentes principais mantidos: {len(pca.explained_variance_ratio_)}")
print(f"Features originais: {features}")


NameError: name 'X_train_scaled' is not defined

## 7. Definição dos Modelos de Regressão

In [8]:
# Definir os 3 modelos para comparação
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=15, 
        min_samples_leaf=2, 
        max_features='sqrt', 
        random_state=42, 
        n_jobs=-1
    ),
    'Decision Tree': DecisionTreeRegressor(
        max_depth=15, 
        min_samples_leaf=2, 
        ccp_alpha=0.0, 
        random_state=42
    )
}

print("Modelos definidos:")
for i, model_name in enumerate(models.keys(), 1):
    print(f"   {i}. {model_name}")


Modelos definidos:
   1. Linear Regression
   2. Random Forest
   3. Decision Tree


## 8. Treinamento e Avaliação dos Modelos

In [9]:
# Dicionário para armazenar resultados
results = {}

# Treinar e avaliar cada modelo
for model_name, model in models.items():
    print(f"{'='*60}")
    print(f"Modelo: {model_name}")
    print(f"{'='*60}")
    
    # Treinar modelo
    model.fit(X_train_pca, y_train)
    
    # Predições
    y_pred = model.predict(X_test_pca)
    
    # Calcular as 3 métricas obrigatórias
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    
    # Calcular acurácia com tolerância
    tolerance = 0.2  # 20% de tolerância
    accuracy = (abs(y_test - y_pred) / y_test <= tolerance).mean() * 100
    
    # Armazenar resultados
    results[model_name] = {
        'R²': r2,
        'MAE': mae,
        'RMSE': rmse,
        'Acurácia (±20%)': accuracy
    }
    
    # Exibir métricas
    print(f"R² Score: {r2:.4f}")
    print(f"MAE (Mean Absolute Error): R$ {mae:.2f}")
    print(f"RMSE (Root Mean Squared Error): R$ {rmse:.2f}")
    print(f"Predições dentro de ±20%: {accuracy:.2f}%")
    print()

print("Todos os modelos foram treinados e avaliados!")


Modelo: Linear Regression
R² Score: 0.5080
MAE (Mean Absolute Error): R$ 5.96
RMSE (Root Mean Squared Error): R$ 11.15
Predições dentro de ±20%: 48.93%

Modelo: Random Forest
R² Score: 0.6911
MAE (Mean Absolute Error): R$ 4.26
RMSE (Root Mean Squared Error): R$ 8.84
Predições dentro de ±20%: 67.63%

Modelo: Decision Tree
R² Score: 0.5880
MAE (Mean Absolute Error): R$ 4.69
RMSE (Root Mean Squared Error): R$ 10.20
Predições dentro de ±20%: 66.25%

Todos os modelos foram treinados e avaliados!


## 9. Comparação dos Modelos

In [10]:
# Tabela comparativa dos modelos
print(f"{'='*70}")
print("COMPARAÇÃO DOS MODELOS")
print(f"{'='*70}")
print(f"{'Modelo':<25} {'R²':<10} {'MAE':<12} {'RMSE':<12} {'Acurácia':<10}")
print(f"{'-'*70}")

for model_name, metrics in results.items():
    print(f"{model_name:<25} {metrics['R²']:<10.4f} R$ {metrics['MAE']:<9.2f} R$ {metrics['RMSE']:<9.2f} {metrics['Acurácia (±20%)']:<9.2f}%")

# Identificar o melhor modelo baseado em R²
best_model = max(results.items(), key=lambda x: x[1]['R²'])
print(f"\n{'='*70}")
print(f"Melhor modelo (por R²): {best_model[0]}")
print(f"   - R² Score: {best_model[1]['R²']:.4f}")
print(f"   - MAE: R$ {best_model[1]['MAE']:.2f}")
print(f"   - RMSE: R$ {best_model[1]['RMSE']:.2f}")
print(f"   - Acurácia (±20%): {best_model[1]['Acurácia (±20%)']:.2f}%")
print(f"{'='*70}")


COMPARAÇÃO DOS MODELOS
Modelo                    R²         MAE          RMSE         Acurácia  
----------------------------------------------------------------------
Linear Regression         0.5080     R$ 5.96      R$ 11.15     48.93    %
Random Forest             0.6911     R$ 4.26      R$ 8.84      67.63    %
Decision Tree             0.5880     R$ 4.69      R$ 10.20     66.25    %

Melhor modelo (por R²): Random Forest
   - R² Score: 0.6911
   - MAE: R$ 4.26
   - RMSE: R$ 8.84
   - Acurácia (±20%): 67.63%


10. Features

In [4]:
# Criar feature de volume do produto
data['time_to_deliver'] = (pd.to_datetime(data['order_delivered_customer_date']) - pd.to_datetime(data['order_purchase_timestamp'])).dt.days

print("Features criadas")

Features criadas


11. Criação das Features

In [5]:
# Definir features para o modelo
features = [
    'price',
    'freight_value'
]

# Preparar X e y (removendo NaN)
X = data[features].dropna()
y = data.loc[X.index, 'review_score']

print(f"Features criadas: {features}")
print(f"Target (Review Score) - média: {y.mean():.2f}")

Features criadas: ['price', 'freight_value']
Target (Review Score) - média: 4.03


12. Train Test Split

In [6]:
# Dividir em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados (necessário para PCA)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Divisão dos dados (20/80)")
print("Normalização aplicada com StandardScaler")

Divisão dos dados (20/80)
Normalização aplicada com StandardScaler


13. PCA - Redução de Dimensionalidade (Obrigatório)

In [7]:
# Aplicar PCA mantendo 95% da variância
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("PCA aplicado:")
print(f"Dimensões originais: {X_train_scaled.shape[1]}")
print(f"Dimensões após PCA: {X_train_pca.shape[1]}")
print(f"\nComponentes principais mantidos: {len(pca.explained_variance_ratio_)}")
print(f"Features originais: {features}")


PCA aplicado:
Dimensões originais: 2
Dimensões após PCA: 2

Componentes principais mantidos: 2
Features originais: ['price', 'freight_value']


14. Definição dos Modelos de Clustering

In [8]:
# Definir os 3 modelos para comparação
models = {
    'K-Means': KMeans(
        init='k-means++',
        n_clusters=4,
    ),
    'DBSCAN': DBSCAN(
        eps=0.8,
        min_samples=5
    ),
    'Agglomerative Clustering': AgglomerativeClustering(
        n_clusters=4,
        linkage='ward'
    )
}

print("Modelos definidos:")
for i, model_name in enumerate(models.keys(), 1):
    print(f"   {i}. {model_name}")

Modelos definidos:
   1. K-Means
   2. DBSCAN
   3. Agglomerative Clustering


15. Treinamento e Avaliação dos Modelos de Clustering

In [ ]:
from collections import Counter

# Treinar e avaliar modelos de clustering
# Usando amostra de 10.000 pontos para reduzir tempo e memória
sample_size = 10000
np.random.seed(42)
sample_indices = np.random.choice(len(X_train_pca), size=min(sample_size, len(X_train_pca)), replace=False)
X_train_sample = X_train_pca[sample_indices]

cluster_results = {}

for model_name, model in models.items():
    print(f"{'='*60}")
    print(f"Modelo: {model_name}")
    print(f"{'='*60}")
    
    # Treinar e obter rótulos na amostra
    labels = model.fit_predict(X_train_sample)
    
    # Informações de clusters
    label_counts = dict(Counter(labels))
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_count = int(label_counts.get(-1, 0))
    
    # Métricas (só calculáveis se houver pelo menos 2 clusters)
    sil_score = None
    ch_score = None
    db_score = None
    if n_clusters > 1:
        try:
            sil_score = metrics.silhouette_score(X_train_sample, labels)
            ch_score = metrics.calinski_harabasz_score(X_train_sample, labels)
            db_score = metrics.davies_bouldin_score(X_train_sample, labels)
        except Exception as e:
            # caso raro de erro numérico
            sil_score = ch_score = db_score = None
    
    # Inércia (apenas KMeans possui)
    inertia = getattr(model, 'inertia_', None)
    
    # Armazenar resultados
    cluster_results[model_name] = {
        'n_clusters': n_clusters,
        'cluster_counts': label_counts,
        'noise_count': noise_count,
        'inertia': float(inertia) if inertia is not None else None,
        'silhouette': float(sil_score) if sil_score is not None else None,
        'calinski_harabasz': float(ch_score) if ch_score is not None else None,
        'davies_bouldin': float(db_score) if db_score is not None else None,
        'labels_sample': labels[:10].tolist()  # amostra dos rótulos
    }
    
    # Exibir métricas resumidas
    print(f"Clusters encontrados (excluindo ruído): {n_clusters}")
    print(f"Contagem por rótulo (inclui -1 para ruído): {label_counts}")
    if inertia is not None:
        print(f"Inertia: {inertia:.4f}")
    print(f"Silhouette: {sil_score if sil_score is not None else 'N/A'}")
    print(f"Calinski-Harabasz: {ch_score if ch_score is not None else 'N/A'}")
    print(f"Davies-Bouldin: {db_score if db_score is not None else 'N/A'}")
    print()

print(f"Avaliação de clustering concluída (amostra de {len(X_train_sample):,} pontos).")
# resultados disponíveis em `cluster_results`

Modelo: K-Means


KeyboardInterrupt: 

## 16. Visualização dos Clusters

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Criar amostra para visualização
# sample_size = 10000
# np.random.seed(42)
# sample_indices = np.random.choice(len(X_train_pca), size=min(sample_size, len(X_train_pca)), replace=False)
X_sample = X_train_pca#[sample_indices]

# Configurar figura com subplots para os modelos
n_models = len(models)
fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 5))
if n_models == 1:
    axes = [axes]
fig.suptitle(f'Visualização dos Clusters - Features: Price + Freight Value (Amostra de {len(X_sample):,} pontos)', fontsize=16, y=1.02)

# Cores para clusters (incluindo ruído como cinza)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8', '#F7DC6F', '#BB8FCE', '#85C1E2']
noise_color = '#808080'

for idx, (model_name, model) in enumerate(models.items()):
    ax = axes[idx]
    
    print(f"Processando {model_name}...")
    
    # Obter labels do clustering na amostra
    labels = model.fit_predict(X_sample)
    
    # Plotar pontos
    unique_labels = set(labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
    
    for label in unique_labels:
        if label == -1:
            # Ruído (DBSCAN)
            color = noise_color
            marker = 'x'
            label_name = 'Ruído'
            alpha = 0.3
        else:
            color = colors[label % len(colors)]
            marker = 'o'
            label_name = f'Cluster {label}'
            alpha = 0.6
        
        mask = labels == label
        ax.scatter(
            X_sample[mask, 0], 
            X_sample[mask, 1] if X_sample.shape[1] > 1 else X_sample[mask, 0],
            c=color,
            label=label_name,
            alpha=alpha,
            edgecolors='black',
            linewidth=0.5,
            marker=marker,
            s=30
        )
    
    # Configurar subplot
    ax.set_title(f'{model_name}\n({n_clusters} clusters)', fontsize=12, fontweight='bold')
    ax.set_xlabel('PCA Component 1 (Price)', fontsize=10)
    ax.set_ylabel('PCA Component 2 (Freight)' if X_sample.shape[1] > 1 else 'PCA Component 1', fontsize=10)
    ax.legend(loc='upper right', fontsize=8, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print("\nVisualizações geradas para todos os modelos de clustering")

Processando K-Means...
Processando DBSCAN...
